In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
mnb = GaussianNB()

## Task 1

1. Load the dataset into a dataframe that can be used for predicting traffic_volume a day in advance.

In [ ]:
#Load the dataset
df = pd.read_csv("metro_traffic_15_19.csv")
df

For the purpose of better training performance, we will implement some simple data cleaning processes.

In [ ]:
#Check the duplicated instances
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())

In [ ]:
#Drop the duplicated instances
df = df.drop_duplicates()
df

In [ ]:
#Check the data types
df.dtypes

In [ ]:
#Convert the 'date_time' into datetime64
df["date_time"] = df["date_time"].astype("datetime64")
df.dtypes

2. Produce some plots at different time-scales to see if there is periodicity in the traffic volume.

In [ ]:
#In order to plot the trendancy of traffic volume based on different time-scales, we create a backup dataframe and make necessary adjustments for plotting.
df_backup = df.copy()

#Extract year, month, weekday and hour from 'date_time'
df_backup["hour"] = df_backup["date_time"].dt.hour
df_backup["year"] = df_backup["date_time"].dt.year
df_backup["month"] = df_backup["date_time"].dt.month
df_backup["weekday"] = df_backup["date_time"].dt.weekday + 1
df_backup

In [ ]:
#We will plot the average traffic volumn of different years using hour(0-23) as base.
#Create a new empty column.
df_backup["avg_traffic_volume_byhour"] = ""

#Create lists to store all the unique values in 'hour' and 'year'.
x_list = df_backup['hour'].unique().tolist()
year_list = df_backup['year'].unique().tolist()

for i in x_list:
    for j in year_list:
        #Calculate the average traffic volumn by hour.
        df_backup["avg_traffic_volume_byhour"].loc[(df_backup['hour']== i) & (df_backup['year']== j)] = df_backup['traffic_volume'].loc[(df_backup['hour']== i) & (df_backup['year']== j)].mean()

In [ ]:
#Check the df_backup
df_backup

In [ ]:
#Convert 'avg_traffic_volume_byhour' to 'float64' data type
df_backup['avg_traffic_volume_byhour'] = df_backup['avg_traffic_volume_byhour'].astype("float64")
df_backup['year'] = df_backup['year'].astype('category')

In [ ]:
#Plot the trendancy using lineplot.
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.lineplot(x=df_backup["hour"], y=df_backup["avg_traffic_volume_byhour"], hue=df_backup["year"])
ax.set(xticks=x_list)
plt.show()

In [ ]:
#Similarly, we plot the average traffic volumn of different years using month(1-12) as base.
df_backup["avg_traffic_volume_bymonth"] = ""
month_list = df_backup['month'].unique().tolist()

for i in month_list:
    for j in year_list:
        df_backup["avg_traffic_volume_bymonth"].loc[(df_backup['month']== i) & (df_backup['year']== j)] = df_backup['traffic_volume'].loc[(df_backup['month']== i) & (df_backup['year']== j)].mean()
        

In [ ]:
df_backup["avg_traffic_volume_bymonth"] = df_backup["avg_traffic_volume_bymonth"].astype("float64")

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.lineplot(x=df_backup["month"], y=df_backup["avg_traffic_volume_bymonth"], hue=df_backup["year"])
ax.set(xticks=month_list)
plt.show()

In [ ]:
#Similarly, we plot the average traffic volumn of different years using month(1-12) as base.
df_backup["avg_traffic_volume_byweekday"] = ""
month_list = df_backup['weekday'].unique().tolist()

for i in month_list:
    for j in year_list:
        df_backup["avg_traffic_volume_byweekday"].loc[(df_backup['weekday']== i) & (df_backup['year']== j)] = df_backup['traffic_volume'].loc[(df_backup['weekday']== i) & (df_backup['year']== j)].mean()
        
df_backup["avg_traffic_volume_byweekday"] = df_backup["avg_traffic_volume_byweekday"].astype("float64")        

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.lineplot(x=df_backup["weekday"], y=df_backup["avg_traffic_volume_byweekday"], hue=df_backup["year"])
ax.set(xticks=month_list)
plt.show()

From the lineplots above, we can see that:
1. Traffic volume varies according to the hour of the day. 3 am has the minimum traffic volum in a day, and 16 pm has the maximum traffic volumn. 
2. From 3 to 7, traffic volumn experiences a constant increase, and from 16 to 3 in the second day, traffic volumn experiences a constant decrease.
3. Trend lines for different years show approximately identical trendancy.
4. The average traffic volume fluctuates over different month of the year. Generally, November, December and Janurary has relatively low traffic volume, while March, June and August has high traffic volume.
5. Monday to Friday have relativley high traffic volume on average and Saturday and Sunday have lower traffic volumn.

## Task 2

1. Extract hour, day and month features from the time-stamps. Build two different regression models and test the accuracy. Try Linear Regression and one other regression model from scikit learn.

2. Divide the data into train and test sets keeping one third of the data for testing.

In [ ]:
#Extract hour, day and month features from the time-stamps.
df["hour"] = df["date_time"].dt.hour
df["day"] = df["date_time"].dt.day
df["month"] = df["date_time"].dt.month
df

In [ ]:
#Divide the data into train and test sets
#We will set the 'traffic_valume' as dependant variable and 'rain_1h', 'snow_1h', 'temp', 'clouds_all', 'hour', 'year', 'day' as independant variables
y = pd.DataFrame(df["traffic_volume"])
X = pd.DataFrame(df[['rain_1h', 'snow_1h', 'temp', 'clouds_all', 'hour', 'month', 'day']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3,random_state=3)


In [ ]:
# need to reset the index to allow contatenation with predicted values otherwise not joining on same index...
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

<b> Linear Regression

In [ ]:
# Train a linear regression model fitting with the training set.
linreg = LinearRegression().fit(X_train, y_train)

In [ ]:
#Check the R squared of training set
print(' R squared statistic of linear regression training set: {:.2f}'.format(linreg.score(X_train, y_train)))

In [ ]:
# calculate the prediction
actual_vs_predicted = pd.concat([y_test, pd.DataFrame(linreg.predict(X_test), columns=['Predicted'])], axis=1)
print(actual_vs_predicted.head(10))

In [ ]:
#Check the R squared of testing set
print(' R squared statistic of linear regression testing set: {:.2f}'.format(linreg.score(X_test, y_test)))

<b> Decision Tree Regression

Here I chose Decision Tree regression model to train the data. The optimal values for paramater ('max_depth' and 'random_state') are not taken into consideration in this assignment.

In [ ]:
#Train the Decision Tree model. 
dtr = DecisionTreeRegressor(max_depth=5, random_state=3)
dtr.fit(X_train, y_train)

In [ ]:
#Check the R squared of training set
print(' R squared statistic of decision tree training set: {:.2f}'.format(dtr.score(X_train, y_train)))

In [ ]:
# actual_vs_predicted = pd.concat([y_test, pd.DataFrame(dtr.predict(X_test), columns=['Predicted'])], axis=1)
# print(actual_vs_predicted.head(10))

In [ ]:
#Check the R squared of testing set
print(' R squared statistic of of decision tree training set: {:.2f}'.format(dtr.score(X_test, y_test)))

### Additional Research

As we were given examples of standarlizing X_train and X_test in 'Regression tutorial' using `StandardScaler()`, in this section I will investigate on whether feature standarlization will impact the accuracy of the linear regression and decision tree model.

In [ ]:
#Standarlize the X datasets
X_train_s = StandardScaler().fit_transform(X_train)
X_test_s = StandardScaler().fit_transform(X_test)

In [ ]:
# Train a linear regression model fitting with the standarlized training set.
linreg_Xs = LinearRegression().fit(X_train_s, y_train)

In [ ]:
#Check the R squared of testing set
print(' R squared statistic of linear regression testing set: {:.2f}'.format(linreg_Xs.score(X_test_s, y_test)))

In [ ]:
#Train the Decision Tree model with the standarlized training set.
dtr_Xs = DecisionTreeRegressor(max_depth=5, random_state=3)
dtr_Xs.fit(X_train_s, y_train)

In [ ]:
#Check the R squared of testing set
print(' R squared statistic of of decision tree training set: {:.2f}'.format(dtr_Xs.score(X_test_s, y_test)))

From the results above we can see that feature Standarlization made no contribution to the improvement of accuracy for both of the linear regression and decision tree model. Therefore, we will no longer consider the feature transformation in the following steps in this assignment.

## Task 3

1. Given that the linear numeric encoding of the hour, day and month features may miss cyclical signals, investigate and test a cyclical strategy for encoding these features. Does this strategy improve accuracy?

In this task, we will investigate cyclical encoding of the hour, day and month features with sine/cosine transformation. 

In task 2, we treat these features as continuous features and put them directly into the model trainning. However, one problem we ignored in task 2 about cyclical features was there are jump discontinuities in the graph at the end of each hour/day/month, for example when the hour value goes from  23  to  00.

A common method for encoding cyclical data is to transform the data into two dimensions using a sine and consine transformation.

We can do that using the following transformations:

𝑥𝑠𝑖𝑛=sin(2∗𝜋∗𝑥max(𝑥)) 

𝑥𝑐𝑜𝑠=cos(2∗𝜋∗𝑥max(𝑥))

In [ ]:
#Here we define two functions to transform hour/day/month data into sine and consine values.
#Code adopted from: https://developer.nvidia.com/blog/three-approaches-to-encoding-time-information-as-features-for-ml-models/ 

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

In [ ]:
#We will copy the initial DataFrame, then encode hour/day/month columns using the sine/cosine transformations.
df2 = df.copy()
df2["hour_sin"] = sin_transformer(24).fit_transform(df2["hour"])
df2["hour_cos"] = cos_transformer(24).fit_transform(df2["hour"])
df2["day_sin"] = sin_transformer(31).fit_transform(df2["day"])
df2["day_cos"] = cos_transformer(31).fit_transform(df2["day"])
df2["month_sin"] = sin_transformer(12).fit_transform(df2["month"])
df2["month_cos"] = cos_transformer(12).fit_transform(df2["month"])
df2

In [ ]:
#We will select the transformed hour/day/month features instead and re-train the linear regression model.
y = pd.DataFrame(df2["traffic_volume"])
X = pd.DataFrame(df2[['rain_1h', 'snow_1h', 'temp', 'clouds_all', 'hour_sin','hour_cos', 'month_sin','month_cos','day_sin','day_cos']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3,random_state=3)

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
linreg = LinearRegression().fit(X_train, y_train)

In [ ]:
actual_vs_predicted = pd.concat([y_test, pd.DataFrame(linreg.predict(X_test), columns=['Predicted'])], axis=1)
print(actual_vs_predicted.head(10))

In [ ]:
print(' R squared statistic of linear regression on training set: {:.2f}'.format(linreg.score(X_train, y_train)))
print(' R squared statistic of linear regression on testing set: {:.2f}'.format(linreg.score(X_test, y_test)))

As we can see above, both the accuracies from training set and testing set experienced significant increases from around 0.15 shown in task 2 to 0.65.

In [ ]:
dtr = DecisionTreeRegressor(max_depth=5, random_state=1)
dtr.fit(X_train, y_train)

In [ ]:
print(' R squared statistic of decision tree on training set: {:.2f}'.format(dtr.score(X_train, y_train)))
print(' R squared statistic of decision tree on testing set: {:.2f}'.format(dtr.score(X_test, y_test)))

However, when it comes to the results of decision tree, we can see that the accuracy based on the data after sine/cosine transformation makes no difference with the data before transformation. The accuracies for training/testing dataset are both around 0.79.

Therefore, we believe that the cyclical strategy of sine/cosine transformation we implemented above for encoding the time feaures such as hour, day and month can significantly improve the accuracy of linear regression model. However, sine/cosine transformation makes minimal contribution for the improvement of performance from decision tree model.

## Task 4

1. Identify subsets of the features for this prediction task. These can be the same subset for all models or model-specific subsets.

Information gain is used for determining the best features/attributes that render maximum information about a class.

In [ ]:
#Code adapted from tutorial 10 Feature Selection
mi = dict()

i_scores = mutual_info_regression(X_train, y_train)

for i,j in zip(X.columns,i_scores):
    mi[i]=j
 
df_subset = pd.DataFrame.from_dict(mi,orient='index',columns=['I-Gain'])
df_subset.sort_values(by=['I-Gain'],ascending=False,inplace=True)

In [ ]:
#Check the IG of each features
df_subset

According to the result in the 'df_subset', we can see that 'hour_cos' and 'hour_sin' are the features with high I-Gain values.

In [ ]:
features = []
for i in df_subset.index.tolist():
    #append i to the feature list
    features.append(i)
    #Split the trainning and testing sest
    y = pd.DataFrame(df2["traffic_volume"])
    X = pd.DataFrame(df2[features])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3,random_state=3)
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    #train the linear regression model
    linreg = LinearRegression().fit(X_train, y_train)
    #Check the accuracy
    print('Feature combination: ', features)
    print(' R squared statistic of linear regression: {:.2f}'.format(linreg.score(X_test, y_test)))

In [ ]:
#Identify the subset for decision tree model
features = []
for i in df_subset.index.tolist():
    #append i to the feature list
    features.append(i)
    #Split the trainning and testing sest
    y = pd.DataFrame(df2["traffic_volume"])
    X = pd.DataFrame(df2[features])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3,random_state=3)
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    #train the linear regression model
    dtr = DecisionTreeRegressor(max_depth=5, random_state=1)
    dtr.fit(X_train, y_train)
    #Check the accuracy
    print('Feature combination: ', features)
    print(' R squared statistic of decision tree: {:.2f}'.format(dtr.score(X_test, y_test)))

From the results above, we can conclude that 'hour_cos' and 'hour_sin' are the most improtant features.

For linear regression model, the model accuracy reached to 0.65 after selecting 'hour_cos' and 'hour_sin' as input features and with later added features are selected, the accuracy shows subtle change.

Decision Tree Regression model show similar situation, the only different is the feature selection of 'hour_cos', 'hour_sin', 'temp' gives a relativly high accuracy of 0.79, while after adding another feature 'clouds_all', the accuracy falls down to 0.78.

### Additional Research

In Task 1, we plotted the relationship between traffic volumn and weekday (the day of the week), and we found out that weekday has strong correlation to average traffic volumn. In this section we will try to consider weekday as the feature and see if it can provide any significant improvement on accuracy.

In [ ]:
#Create 'weekday' feature and transform it using sin/cos transformer.
df2["weekday"] = df2["date_time"].dt.weekday + 1
df2["weekday_sin"] = sin_transformer(7).fit_transform(df2["weekday"])
df2["weekday_cos"] = cos_transformer(7).fit_transform(df2["weekday"])

#We will select 'hour_sin','hour_cos' based on the conclusion we have made, and includes weekday_sin and weekday_cos.
y = pd.DataFrame(df2["traffic_volume"])
X = pd.DataFrame(df2[['hour_sin','hour_cos', "weekday_sin", "weekday_cos"]])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3,random_state=3)

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
#train the linear regression model
linreg = LinearRegression().fit(X_train, y_train)
#Check the accuracy
print(' R squared statistic of linear regression: {:.2f}'.format(linreg.score(X_test, y_test)))

In [ ]:
dtr = DecisionTreeRegressor(max_depth=5, random_state=1)
dtr.fit(X_train, y_train)
#Check the accuracy
print(' R squared statistic of decision tree: {:.2f}'.format(dtr.score(X_test, y_test)))

We can see that 'weekday' feature do provide an important contribution in accuracy improvement for both linear regression (increased from 0.65 to 0.69 on test set) and decision tree regression (increased from 0.78 to 0.92).